In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [ ]:
bone_data <- read_csv("data/boneMarrow.csv")
head(bone_data)
colnames(bone_data)

Rows: 187 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (13): RecipientABO, RecipientRh, ABOmatch, CMVstatus, DonorCMV, Recipien...
dbl (25): id, Recipientgender, Stemcellsource, Donorage, Donorage35, IIIV, G...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,RecipientABO,RecipientRh,⋯,extcGvHD,CD34kgx10d6,CD3dCD34,CD3dkgx10d8,Rbodymass,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time,survival_status
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,22.83014,0,1,0,1,1,1,⋯,1,7.20,1.33876,5.38,35,19,51,32,999,0
2,1,0,23.34247,0,1,0,-1,-1,1,⋯,1,4.50,11.078295,0.41,20.6,16,37,1000000,163,1
3,1,0,26.39452,0,1,0,-1,-1,1,⋯,1,7.94,19.01323,0.42,23.4,23,20,1000000,435,1
4,0,0,39.68493,1,1,0,1,2,1,⋯,?,4.25,29.481647,0.14,50,23,29,19,53,1
5,0,1,33.35890,0,0,0,1,2,0,⋯,1,51.85,3.972255,13.05,9,14,14,1000000,2043,0
6,1,0,27.39178,0,0,0,2,0,1,⋯,1,3.27,8.412758,0.39,40,16,70,1000000,2800,0


[1] "id"                   "Recipientgender"      "Stemcellsource"      
 [4] "Donorage"             "Donorage35"           "IIIV"                
 [7] "Gendermatch"          "DonorABO"             "RecipientABO"        
[10] "RecipientRh"          "ABOmatch"             "CMVstatus"           
[13] "DonorCMV"             "RecipientCMV"         "Disease"             
[16] "Riskgroup"            "Txpostrelapse"        "Diseasegroup"        
[19] "HLAmatch"             "HLAmismatch"          "Antigen"             
[22] "Alel"                 "HLAgrI"               "Recipientage"        
[25] "Recipientage10"       "Recipientageint"      "Relapse"             
[28] "aGvHDIIIIV"           "extcGvHD"             "CD34kgx10d6"         
[31] "CD3dCD34"             "CD3dkgx10d8"          "Rbodymass"           
[34] "ANCrecovery"          "PLTrecovery"          "time_to_aGvHD_III_IV"
[37] "survival_time"        "survival_status"

In [ ]:
bone_selected <- bone_data |>
select(Donorage, Recipientage, DonorABO, RecipientABO, Relapse, survival_time, survival_status)|>
mutate(survival_status = as_factor(survival_status))
bone_selected

Donorage,Recipientage,DonorABO,RecipientABO,Relapse,survival_time,survival_status
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<fct>
22.83014,9.6,1,1,0,999,0
23.34247,4.0,-1,-1,1,163,1
26.39452,6.6,-1,-1,1,435,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮
27.34794,10.4,1,-1,0,1895,0
27.78082,8.0,1,0,0,382,1
55.55342,9.5,1,2,0,1109,0


In [ ]:
survival_summarize <- bone_selected |>
group_by(survival_status) |>
summarize(count=n())
survival_summarize

survival_status,count
<fct>,<int>
0,102
1,85


In [ ]:
set.seed(1)
bone_split <- initial_split(bone_selected, prop=0.7, strata=survival_status)
bone_train <- training(bone_split)
bone_test <- testing(bone_split)
train_summarize <- bone_train |>
group_by(survival_status) |>
summarize(count=n())
train_summarize

survival_status,count
<fct>,<int>
0,71
1,59
